## Numerical Linear Algebra
## Homework_1 Exercise_2

### Libraries importation

In [17]:
import numpy
import scipy.linalg

from scipy.sparse import diags
import numpy as np
import math

### Tridiagonal matrix corresponding to the system

In [31]:
n = 4
b = 1; a = 0
h =(b-a)/(n+1)
Z = np.linspace(a, b, n+1)
P = np.ones(n+1)
Q = np.ones(n+1)
b_h = np.ones(n+1)
v = np.ones(n+1)
for i in range(len(Z)):
    P[i] = math.pow(Z[i],2)
    Q[i] = math.exp(-Z[i])
    b_h[i] = math.exp(-P[i])*(h**2)
    v[i] = -(2 + h*P[i] + (h**2)*Q[i])

u = np.ones(n)
w = np.ones(n)
for i in range(n):
    u[i] = 1 + h*P[i]

print(u)
print(v)
print(w)
print(b_h)

def tridiag(u, v, w, k1=-1, k2=0, k3=1):
    return np.diag(u, k1) + np.diag(v, k2) + np.diag(w, k3)

A = tridiag(u, v, w)
A

[1.     1.0125 1.05   1.1125]
[-2.04       -2.04365203 -2.07426123 -2.13139466 -2.21471518]
[1. 1. 1. 1.]
[0.04       0.03757652 0.03115203 0.02279131 0.01471518]


array([[-2.04      ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        , -2.04365203,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.0125    , -2.07426123,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  1.05      , -2.13139466,  1.        ],
       [ 0.        ,  0.        ,  0.        ,  1.1125    , -2.21471518]])

In [47]:
M = A
syms = b_h

### Gaussian elimination method

In [48]:

# write rows in row echelon form
def upper_triangular(M):
    # move all zeros to buttom of matrix
    M = np.concatenate((M[np.any(M != 0, axis=1)], M[np.all(M == 0, axis=1)]), axis=0)

    # iterate over matrix rows
    for i in range(0, M.shape[0]):

        # initialize row-swap iterator
        j = 1

        # select pivot value
        pivot = M[i][i]

        # find next non-zero leading coefficient
        while pivot == 0 and i + j < M.shape[0]:
            # perform row swap operation
            M[[i, i + j]] = M[[i + j, i]]

            # incrememnt row-swap iterator
            j += 1

            # get new pivot
            pivot = M[i][i]

        # if pivot is zero, remaining rows are all zeros
        if pivot == 0:
            # return upper triangular matrix
            return M

        # extract row
        row = M[i]

        # get 1 along the diagonal
        M[i] = row / pivot

        # iterate over remaining rows
        for j in range(i + 1, M.shape[0]):
            # subtract current row from remaining rows
            M[j] = M[j] - M[i] * M[j][i]

    # return upper triangular matrix
    return M

In [51]:
upper_triangular(M)

array([[ 1.        , -0.49019608, -0.        , -0.        , -0.        ],
       [-0.        ,  1.        , -0.64372601, -0.        , -0.        ],
       [-0.        , -0.        ,  1.        , -0.70299331, -0.        ],
       [-0.        , -0.        , -0.        ,  1.        , -0.71774541],
       [-0.        , -0.        , -0.        , -0.        ,  1.        ]])

In [52]:
scipy.linalg.solve(M, b_h)

array([0.51950932, 0.76918274, 0.996394  , 0.65422943, 0.3256239 ])

In [34]:
def backsubstitution(M, syms):
    # symbolic variable index
    for i, row in reversed(list(enumerate(M))):
        # create symbolic equation
        eqn = -M[i][-1]
        for j in range(len(syms)):
            eqn += syms[j] * row[j]

        # solve symbolic expression and store variable
        syms[i] = scipy.linalg.solve(eqn, syms[i])[0]

    # return list of evaluated variables
    return syms

backsubstitution(M, syms)

array([-0.29061627, -0.05603854, -0.6337545 , -0.02258351,  0.00666605])

### Resolution using QR decomposition

In [35]:
Q, R = scipy.linalg.qr(A)

In [43]:
vector_b = np.dot(Q.transpose(), b_h.transpose()) 
vector_b

array([ 0.23628461, -0.22804397,  0.44107078,  0.29887723, -0.31319618])

In [44]:
scipy.linalg.solve(R, vector_b)

array([0.51950932, 0.76918274, 0.996394  , 0.65422943, 0.3256239 ])

### We noticed that we got almost the same solution using Gaussian elimation and the QR methods